[![Open in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eagalpern/folding-ising-globular/blob/master/notebooks/simulation_colab.ipynb)



# Folding Ising Simulation for any sequence of 15 protein families

Demonstration Notebook for "How natural sequence variation modulates protein folding dynamics"

**Authors**: Ezequiel A. Galpern, Ernesto A. Roman, Diego U. Ferreiro

**DOI**: [10.48550/arXiv.2412.14341](https://doi.org/10.48550/arXiv.2412.14341)



In [1]:
# @title Import data and install dependencies

import pandas as pd
from matplotlib import pyplot as plt, colors
import numpy as np
import os
import zipfile
import requests
import subprocess
import pickle
import sys
import seaborn as sns
import json
from multiprocessing import cpu_count
import importlib.util
from ipywidgets import interact, widgets, Checkbox, Text, VBox, Output
from IPython.display import display, clear_output

# Install py3Dmol for visualizing structures and Bio to load fasta sequences
def install_if_missing(package, import_name=None):
    if import_name is None:
        import_name = package
    if importlib.util.find_spec(import_name) is None:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", package])
install_if_missing("py3Dmol")
install_if_missing("biopython", "Bio")
#!pip install --quiet py3Dmol Bio

# Detect if running in Google Colab
IN_COLAB = 'COLAB_GPU' in os.environ

# Set the repository URL, clone directory, and data directory
REPO_URL = "https://github.com/eagalpern/folding-ising-globular.git"
SOURCE_DIR = "/content/repo" if IN_COLAB else "../"
DATA_DIR = os.path.join(SOURCE_DIR, "data")


# Set up Zenodo URL
ZENODO_ID = "14547875"  # Replace with your actual Zenodo DOI
ZIP_FILE_URL = f"https://zenodo.org/records/{ZENODO_ID}/files/simplified_rbm_and_msa.zip?download=1"
ZENODO_DATA_DIR = "/content/simplified_rbm_and_msa"  # This will be the extracted folder


# Clone the GitHub repository in Colab only
if IN_COLAB:
    if not os.path.exists(SOURCE_DIR):
        subprocess.run(f'git clone {REPO_URL} {SOURCE_DIR}', shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    sys.path.append(f"{SOURCE_DIR}/source/")

    # from google.colab import drive
    # # Mount Google Drive
    # drive.mount('/content/drive')
    # GDRIVE_PATH = "/content/drive/Shareddrives/Ezequiel/dca_frustration/simplified_rbm_and_msa"

    # Download and unzip the Zenodo data
    print(f"Downloading data from Zenodo: {ZIP_FILE_URL}")
    zip_file_path = "/content/simplified_rbm_and_msa.zip"
    response = requests.get(ZIP_FILE_URL)
    with open(zip_file_path, 'wb') as f:
        f.write(response.content)

    # Unzip the data
    print(f"Unzipping the data...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall("/content")

    # Verify extraction and check data
    if os.path.exists(ZENODO_DATA_DIR):
        print(f"Zenodo Data extracted successfully to {ZENODO_DATA_DIR}")
    else:
        print("Extraction failed!")

else:
    # For local usage
    sys.path.append('../source/')
    SOURCE_DIR = '/home/ezequiel/libraries/folding-ising-globular/'
    DATA_DIR = os.path.join(SOURCE_DIR, "data")

from fasta_utils import *
from visualization import *
from plot_results import *
from ising_simulation import *
from utils import *
from DMS_prediction import *

table_file = os.path.join(DATA_DIR, "table_s1.csv")
tsel_fam = pd.read_csv(table_file, index_col =0)
tsel_fam['foldon_breaks']= tsel_fam.foldon_breaks.apply(lambda x: np.array(json.loads(x)))
tsel_fam['foldon_breaks_pdb']= tsel_fam.foldon_breaks_pdb.apply(lambda x: np.array(json.loads(x)))

Unzipping the data...
Zenodo Data extracted successfully to /content/simplified_rbm_and_msa


In [2]:
# @title Folding Simulation

def combined_widget_with_simulation():
    # Output widget for displaying messages
    output = Output()

    # Separate output widget for 3D visualization
    pdb_output = Output()


    # Dropdown to select the family, default set to 'DHFR'
    family_dropdown = widgets.Dropdown(
        options=tsel_fam.Family.tolist(),
        value='DHFR',  # Initial value
        description='Family:',
        style={'description_width': 'initial'}
    )

    # Checkbox for "Reference Sequence"
    ref_sequence_checkbox = widgets.Checkbox(
        value=False,
        description="Reference Sequence",
        style={'description_width': 'initial'}
    )

    # Text input for UniProt ID
    uniprot_id_input = widgets.Text(
        value='',
        description="UniProt ID:",
        style={'description_width': 'initial'}
    )

    # Text input for the sequence
    sequence_input = widgets.Text(
        value='',
        description="Sequence:",
        style={'description_width': 'initial'}
    )

    # Text input for Protein Name
    prot_name_input = widgets.Text(
        value='',
        description="Protein Name:",
        style={'description_width': 'initial'}
    )

    # Action buttons
    process_button = widgets.Button(description="Search Sequence")
    confirm_button = widgets.Button(description="Confirm Sequence", disabled=True)
    simulation_button = widgets.Button(description="Run Simulation", disabled=True)

    # Widgets for vmin, vmax, and cp_factor
    vmin_widget = widgets.IntSlider(
        value=100,
        min=10,
        max=500,
        step=10,
        description='Min temperature:',
        continuous_update=False,
        style={'description_width': 'initial'}
    )

    vmax_widget = widgets.IntSlider(
        value=500,
        min=50,
        max=1000,
        step=10,
        description='Max temperature:',
        continuous_update=False,
        style={'description_width': 'initial'}
    )

    cp_factor_widget = widgets.IntSlider(
        value=20,
        min=1,
        max=10000,
        step=1,
        description='Critical Points factor:',
        continuous_update=False,
        style={'description_width': 'initial'}
    )




    # Variables to store family-related information
    family_data = {"FAMILY_PATH": None, "FASTA_PATH": None, "i": None, "seq_len": None}

    # Function to update the family-related data
    def update_family(change=None):
        family_name = family_dropdown.value
        i = tsel_fam.loc[tsel_fam.Family == family_name].index[0]
        family = tsel_fam.family[i]
        seq_len = tsel_fam.seq_len[i]  # Sequence length


        FAMILY_PATH = os.path.join(ZENODO_DATA_DIR, family)

        fasta_file = get_fasta_file(FAMILY_PATH)
        FASTA_PATH = os.path.join(FAMILY_PATH, fasta_file)

        # Update family data
        family_data.update({"FAMILY_PATH": FAMILY_PATH, "FASTA_PATH": FASTA_PATH, "i": i, "seq_len": seq_len})

        # Automatically update UniProt ID and Sequence if "Reference Sequence" is checked
        if ref_sequence_checkbox.value:
            update_uniprot_id_and_sequence()

        # Display family-related information
        with output:
            output.clear_output()
            print(f"Family: {family_name}")
            print(f"Expected Sequence Length: {seq_len}")

    # Function to update UniProt ID and Sequence
    def update_uniprot_id_and_sequence():
        if family_data["i"] is not None:  # Ensure family is selected
            i = family_data["i"]
            uniprot_id_input.value = tsel_fam["target_seq"][i].split('.')[0]
            seq_ = extract_sequence_from_fasta(uniprot_id_input.value, family_data["FASTA_PATH"])
            sequence_input.value = ''.join(seq_) if seq_ else ''

    # Attach the update function to the checkbox
    ref_sequence_checkbox.observe(lambda _: update_uniprot_id_and_sequence(), 'value')

    # Function to process the sequence
    def process_sequence(_):
        with output:
            output.clear_output()
            if family_data["FASTA_PATH"] is None:
                print("Error: Please select a family first.")
                return

            uniprot_id = uniprot_id_input.value.strip()
            sequence = sequence_input.value.strip()

            if not uniprot_id and not sequence:
                print("Error: Please provide either a UniProt ID or a sequence.")
                return
            
            # If UniProt ID is provided, extract the sequence from the FASTA file
            if uniprot_id and not sequence:
                seq_ = extract_sequence_from_fasta(uniprot_id, family_data["FASTA_PATH"])
                if seq_:
                    sequence_input.value = ''.join(seq_)  # Populate the sequence field
                    sequence = sequence_input.value.strip()  # Update the sequence variable
                else:
                    print(f"Error: No sequence found for UniProt ID {uniprot_id}.")
                    return
                
            # Check if the sequence length matches the expected length
            if len(sequence) != family_data["seq_len"]:
                print(f"Error: Sequence length must be {family_data['seq_len']} but got {len(sequence)}.")
                return

            confirm_button.disabled = False  # Enable the confirm button once sequence is valid
            print("Sequence processed successfully.")

    # Attach the action to the process button
    process_button.on_click(process_sequence)

    # Shared variable to store the confirmed sequence
    confirmed_sequence = [None]  # Use a mutable list to avoid global issues

    # Function to confirm the sequence
    def confirm_sequence(_):
        with output:

          output.clear_output()
          sequence = sequence_input.value.strip()
          uniprot_id = uniprot_id_input.value.strip()

          # Extract sequence from FASTA
          seq_ = extract_sequence_from_fasta(uniprot_id, family_data["FASTA_PATH"])
          old_seq = ''.join(seq_)
          if old_seq == sequence:
              prot_name_input.value = uniprot_id
              confirmed_sequence[0] = sequence  # Set the confirmed sequence
          elif len(old_seq) == len(sequence):
              prot_name_input.value = uniprot_id + '_mutant'
              confirmed_sequence[0] = sequence  # Use the potentially modified sequence
          else:
              print(f"Error: Sequence length must be {len(old_seq)} but got {len(sequence)}.")
              simulation_button.disabled = True
              return
          # Enable the simulation button
          simulation_button.disabled = False

        # Debug: Check the assigned value
        with output:
            print(f"Confirmed Sequence: {confirmed_sequence[0]}")
    confirm_button.on_click(confirm_sequence)


    # Function to run the simulation and handle errors
    def run_simulation(_):
        with output:
            output.clear_output(wait=True)
            print('The simulation may take a few minutes, please wait.')

            vmin = vmin_widget.value  # Get the vmin value from the widget
            vmax = vmax_widget.value  # Get the vmax value from the widget
            num_cores = cpu_count()

            i = family_data["i"]
            Tsel = tsel_fam.Tsel[i]
            breaks = tsel_fam.foldon_breaks[i]
            potts = np.load(os.path.join(family_data["FAMILY_PATH"], "potts.npz"))

            pdb_code = tsel_fam.pdb_code[i]

            # Ensure the confirmed sequence is available
            if not confirmed_sequence[0]:
                print("Error: No confirmed sequence available.")
                return

            # Ensure all required inputs are provided
            if not sequence_input.value.strip() or not prot_name_input.value.strip():
                print("Error: Sequence or Protein Name is missing.")
                return

            # Run the simulation
            try:
                output_dir = 'results/'
                os.makedirs(output_dir, exist_ok=True)

                # Pass the confirmed sequence to the simulation
                features = ising_simulation(
                    potts=potts,
                    breaks=breaks,
                    folder=output_dir,
                    seq=confirmed_sequence[0],  # Use the confirmed sequence
                    prot_name=prot_name_input.value,
                    Tsel=Tsel,
                    si0=0.005,  # Same as in ank paper
                    k=0.001985875,  # [kcal /(mol K)]
                    tini_=vmin,
                    tfin_=vmax,
                    DT=10,
                    cp_factor=cp_factor_widget.value,
                    interactions_off=False
                )
                print("Simulation completed successfully.")

                # Call visualization functions only after simulation is successful
                plot_results_and_visualizations(output_dir, prot_name_input.value, features,vmin,vmax,pdb_code,i,breaks)

            except Exception as e:
                print(f"Error during simulation: {e}")

    # Function to handle all plotting/visualization tasks
    def plot_results_and_visualizations(output_dir, prot_name, features, vmin,vmax,pdb_code,i,breaks ):
        seq_len = len(confirmed_sequence[0])

        try:

            # Check if plot_results should be displayed
           # if plot_results:
            fig, ax = build_axes_2(1, 20, 5.5)
            ax_ff = ax[0]
            ax_domains_and_fe = [ax[1], ax[2], ax[3]]
            colors_d = plot_ising(
                output_dir, ax_ff, ax_domains_and_fe, prot_name=prot_name, num_cores=cpu_count(),
                vmin=vmin, vmax=vmax, lw=0.6, lw_fq=1.5, alpha_fq=1, inter_t=1,
                fontsize=10, noninf=False, t0=50)
            plt.show()
# Check if view3d visualization should be displayed

            #if view3d:
            with pdb_output:
              pdb_output.clear_output(wait=True)
              t_ = load_features(output_dir + prot_name + '/')['t_']

              # PDB
              path_pdb = os.path.join(DATA_DIR, 'pdb_files')
              pdb_file = path_pdb + '/' + pdb_code + '_cleaned.pdb'
              ali_seq_num_pdb = np.arange(tsel_fam.pdb_ali_beg[i], tsel_fam.pdb_ali_end[i] + 1)

              temps_seq_ref, colors_seq_ref = map_t_seq_3d(
                  t_, breaks, seq_len, rgb=True, vmin=vmin, vmax=vmax)

              view = view_3d_exon_hist(
                  ali_seq_num_pdb, [rgb2hex(c) for c in colors_seq_ref / 255], pdb_file)

            # Check if plot_dssp visualization should be displayed
#            if plot_dssp:
            path_dssp = os.path.join(DATA_DIR, 'dssp')
            dssp_data = pd.read_csv(path_dssp + '/' + pdb_code + '.csv')
            if pdb_code=='7dfr':
              positions=dssp_data.exon_freq
            else:
              positions = ~np.isnan(dssp_data.exon_freq)

            temps_seq_ref, colors_seq_ref = map_t_seq_3d(
                t_, breaks, seq_len, rgb=True, vmin=vmin, vmax=vmax)

            fig, ax = plt.subplots(1, figsize=(8, 0.5))
            colors_pdb = np.array([[255.0, 255.0, 255.0]] * len(positions))
            colors_pdb[positions] = colors_seq_ref
            plot_ss_(dssp_data, colors_pdb / 255.0, ax)
            ax.set_axis_off()
            plt.show()
            view.show()

        except Exception as e:
            print(f"Error during visualization: {e}")


    # Attach the simulation action to the simulation button
    simulation_button.on_click(run_simulation)



    # Attach the update function to the family dropdown
    family_dropdown.observe(update_family, 'value')

    # Call update_family explicitly to process the default value
    update_family()

    # Display all widgets together
    display(VBox([
        family_dropdown,
        ref_sequence_checkbox,
        uniprot_id_input,
        process_button,
        sequence_input,
        confirm_button,
        prot_name_input,
        vmin_widget,
        vmax_widget,
        cp_factor_widget,
        simulation_button,
        output,
        pdb_output
    ]))
    simulation_button.on_click(run_simulation)


# Call the updated widget
combined_widget_with_simulation()


In [ ]:
# @title Cooperativity predictions for single-site mutants
coop_predictions = None

def analysis_widget():
    global coop_predictions  # Declare coop_predictions as global

    # Output widget for displaying messages
    output = Output()

    # Dropdown to select the family, default set to 'DHFR'
    family_dropdown = widgets.Dropdown(
        options=tsel_fam.Family.tolist(),
        value='DHFR',  # Initial value
        description='Family:',
        style={'description_width': 'initial'}
    )

    # Checkbox for "Reference Sequence"
    ref_sequence_checkbox = widgets.Checkbox(
        value=False,
        description="Reference Sequence",
        style={'description_width': 'initial'}
    )

    # Text input for UniProt ID
    uniprot_id_input = widgets.Text(
        value='',
        description="UniProt ID:",
        style={'description_width': 'initial'}
    )

    # Text input for the sequence
    sequence_input = widgets.Text(
        value='',
        description="Sequence:",
        style={'description_width': 'initial'}
    )

    # Action buttons
    process_button = widgets.Button(description="Process Sequence")
    analyze_button = widgets.Button(description="Run Analysis", disabled=True)

    # Variables to store family-related information
    family_data = {"FAMILY_PATH": None, "FASTA_PATH": None, "i": None, "seq_len": None}

    # Function to update the family-related data
    def update_family(change=None):
        family_name = family_dropdown.value
        i = tsel_fam.loc[tsel_fam.Family == family_name].index[0]
        family = tsel_fam.family[i]
        seq_len = tsel_fam.seq_len[i]  # Sequence length

        FAMILY_PATH = os.path.join(ZENODO_DATA_DIR, family)
        fasta_file = get_fasta_file(FAMILY_PATH)
        FASTA_PATH = os.path.join(FAMILY_PATH, fasta_file)

        # Update family data
        family_data.update({"FAMILY_PATH": FAMILY_PATH, "FASTA_PATH": FASTA_PATH, "i": i, "seq_len": seq_len})

        # Automatically update UniProt ID and Sequence if "Reference Sequence" is checked
        if ref_sequence_checkbox.value:
            update_uniprot_id_and_sequence()

        # Display family-related information
        with output:
            output.clear_output()
            print(f"Family: {family_name}")
            print(f"Expected Sequence Length: {seq_len}")

    # Function to update UniProt ID and Sequence
    def update_uniprot_id_and_sequence():
        if family_data["i"] is not None:  # Ensure family is selected
            i = family_data["i"]
            uniprot_id_input.value = tsel_fam["target_seq"][i].split('.')[0]
            seq_ = extract_sequence_from_fasta(uniprot_id_input.value, family_data["FASTA_PATH"])
            sequence_input.value = ''.join(seq_) if seq_ else ''

    # Attach the update function to the checkbox
    ref_sequence_checkbox.observe(lambda _: update_uniprot_id_and_sequence(), 'value')

    # Function to process the sequence
    def process_sequence(_):
        with output:
            output.clear_output()
            if family_data["FASTA_PATH"] is None:
                print("Error: Please select a family first.")
                return

            uniprot_id = uniprot_id_input.value.strip()
            sequence = sequence_input.value.strip()

            if not uniprot_id and not sequence:
                print("Error: Please provide either a UniProt ID or a sequence.")
                return

            # If UniProt ID is provided, extract the sequence from the FASTA file
            if uniprot_id and not sequence:
                seq_ = extract_sequence_from_fasta(uniprot_id, family_data["FASTA_PATH"])
                if seq_:
                    sequence_input.value = ''.join(seq_)  # Populate the sequence field
                    sequence = sequence_input.value.strip()  # Update the sequence variable
                else:
                    print(f"Error: No sequence found for UniProt ID {uniprot_id}.")
                    return

            # Check if the sequence length matches the expected length
            if len(sequence) != family_data["seq_len"]:
                print(f"Error: Sequence length must be {family_data['seq_len']} but got {len(sequence)}.")
                return

            analyze_button.disabled = False  # Enable the analyze button once sequence is valid
            print("Sequence processed successfully.")

    # Function to run the analysis
    def run_analysis(_):
        global coop_predictions  # Declare coop_predictions as global
        with output:
            output.clear_output()
            sequence = sequence_input.value.strip()
            if not sequence:
                print("Error: No sequence available.")
                return
            print('Cooperativity predictions may take a few seconds, please wait.')

            # Get family data
            i = family_data["i"]
            potts = np.load(os.path.join(family_data["FAMILY_PATH"], "potts.npz"))
            breaks = tsel_fam.foldon_breaks[i]

            # Convert sequence to uppercase
            seq = np.array([np.char.upper(x) for x in sequence])
            seq_len = len(seq)

            # Foldons
            seq_ix = tsel_fam.foldon_breaks_pdb[i].copy()
            seq_ix[-1] = seq_ix[-1] - 1

            # Load cooperativity coefficients
            coef = pd.read_csv(os.path.join(DATA_DIR, 'cooperativity_fit.csv'), index_col=0)

            # Compute energy averages
            k = 0.001985875
            m = -1 / (k * tsel_fam.Tsel[i])
            AAdict = {
                'Z': 4, 'X': 0, '-': 0, 'B': 3, 'J': 8, 'A': 1, 'C': 2, 'D': 3,
                'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11,
                'N': 12, 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'V': 18,
                'W': 19, 'Y': 20
            }

            decoy_es_mean, decoy_dif_ei_mean = compute_decoy_energy_averages(seq, potts, breaks, m, AAdict)
            es_mean_wt, dif_ei_mean_wt = compute_energy_averages(seq, potts, breaks, m, AAdict)

            # Plot results
            fig, ax = plt.subplots(1, figsize=(int(len(seq) / 5), 3.8))
            fontsize = 12
            tick_size = 10
            num_size = 8
            xtick_sample = 10

            prediction_mut, prediction_wt, error = make_prediction_plots(
                ax, coef, dif_ei_mean_wt, es_mean_wt, decoy_dif_ei_mean, decoy_es_mean, seq_ix, xtick_sample, fontsize, tick_size, num_size
            )

            # Create mutational table
            coop_predictions = create_mutational_table(prediction_mut, list(seq), 'cooperativity', tsel_fam.pdb_ali_beg[i])
            coop_predictions.loc[coop_predictions['cooperativity_mutant'] > 1, 'cooperativity_mutant'] = 1
            coop_predictions.loc[coop_predictions['cooperativity_mutant'] < 0, 'cooperativity_mutant'] = 0
            output.clear_output()

            # Display the plot
            plt.show()

            print("Analysis completed. The cooperativity predicitons are stored in the variable `coop_predictions`.")

    # Attach actions to buttons
    process_button.on_click(process_sequence)
    analyze_button.on_click(run_analysis)

    # Attach the update function to the family dropdown
    family_dropdown.observe(update_family, 'value')

    # Call update_family explicitly to process the default value
    update_family()

    # Display all widgets together
    display(VBox([
        family_dropdown,
        ref_sequence_checkbox,
        uniprot_id_input,
        sequence_input,
        process_button,
        analyze_button,
        output
    ]))


# Call the analysis widget
analysis_widget()

In [ ]:
coop_predictions.sort_values('cooperativity_difference').head(10) 